<a href="https://colab.research.google.com/github/oladimejiala/MSSA1-Data/blob/main/MSSA_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Loading the python Libraries***

In [ ]:
import pandas_datareader as web
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import MinMaxScaler, Binarizer, Normalizer, LabelBinarizer  
import seaborn as sns
import requests

# **Data Bembereke Participant's Data from My Google Drive**

In [ ]:
# # plotting stock price data
# def test_run():
#     df = pd.read_csv('https://api.ona.io/api/v1/export/2216667.csv?temp_token=128d825e31f02c66392258009f2647dc06552ee9')
#     # df = pd.read_csv("data/AAPL.csv")
#     print (df[''])
#     df['Adj Close'].plot()
#     plt.show() #must be called to show plots
#     sns.pairplot()

# if __name__== "__main__":
#     test_run()

#**Data visualization : Pairwise plotting of biometric parameters**

In [ ]:
# bem = pd.read_csv('/content/Bembereke Cleaned Data 14th April.csv')
# print(bem)
bem = pd.read_csv('/content/newMSSA.csv')

In [ ]:
bem.head()

# **Pairplot of Biometric features**: height, muac, weight, waist-circumference.

In [ ]:
biometrics =  bem[['height(cm)','waist_circum2(cm)', 'weight(kg)', 'MUAC(cm)']]
sns.pairplot(biometrics)

In [ ]:
#this is a combined biometrics to be used against anemia in combination with STH results
x_biometrics = bem.iloc[:, [7,8,9,10,11,34]].values  #'height(cm)','MUAC(cm)','weight(kg)', 'waist_circ', pcv2
# from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
x_biometrics = mms.fit_transform(x_biometrics)
x_b = x_biometrics[:,:5]
# bio = x_biometrics[:,:5]

Multiple Linear Regression Model between Anthropometrics and Packed Cell Volume

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
lr = LinearRegression()
pcv = x_biometrics[:,5:]#the last column representing the pcv
x_bio_train, x_bio_test, y_pcv_train, y_pcv_test = train_test_split(bio,pcv, test_size=0.2, random_state=42)

model_bio_pcv = lr.fit(x_bio_train, y_pcv_train)

y_pcv_pred = model_bio_pcv.predict(x_bio_test)
pcv_bio_pred_score = model_bio_pcv.score(x_bio_test, y_pcv_test)

lr.coef_ #Regression coeficient

# plt.scatter(x_bio_train, y_pcv_train)
plt.hist(y_pcv_pred)
# Note that ypu cannot plot the regression due to inconsistency in the number of samples and size.


In [ ]:
psac = bem.Age_Group
# age_sac = bem['Age_SAC']
# sns.pairplot(age_psac,age_sac)
# psac_new = age_psac.copy()                             # Create copy of DataFrame
# psac_new = psac_new.fillna(psac_new.mean())            # Mean imputation
# print(psac_new)                                        # Print updated DataFrame

# ***Correlations between features***

There is strong correlation between participants height and waist circumference(0.7282), weight and waist circumference ( 0.7321), 

In [ ]:
biometrics.corr()

# Body Mass Index of Participants 
---

In [ ]:
height_metres = bem['height(cm)']/100
bmi = biometrics['weight(kg)']/(height_metres)**2 # this is the body mass index of both preschool and schoo age children
MUAC = biometrics['MUAC(cm)'].values
sns.regplot(bmi, MUAC)
plt.xlabel('Body Mass Index')
plt.title('Body Mass Index vs MUAC')
plt.ylabel('Mid Upper Arm Circumference')
print(" This is the mean Body Mass index:", np.mean(bmi))

# **Age, Sex, BMI, MUAC**

In [ ]:
MUAC_df = biometrics['MUAC(cm)']
waist_circ = biometrics['waist_circum2(cm)']
waist_circ.corr(MUAC_df)

# plt.scatter(waist_circ, MUAC_df)

# waist_muac = [[waist_circ, MUAC_df]]

x_waist = waist_circ.values.reshape(-1, 1)
y_MUAC_df = MUAC_df.values

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
x_train, x_test, y_train, y_test = train_test_split(x_waist, y_MUAC_df, test_size = 0.2, random_state = 0)

lr = LinearRegression()
model = lr.fit(x_train, y_train)
y_pred = model.predict(x_test)
sns.regplot(y_pred, y_test, marker='*', color= 'red')
plt.title('Fig: Waist Circumference vs MUAC')
plt.xlabel('Waist Circumference')
plt.ylabel('MUAC')

In [ ]:
y_pred = lr.predict(x_test)
y_pred
# plt.plot(x_train, lr.predict(x_train), color = 'blue')
# plt.scatter(x_test, y_test, color = 'red')
plt.xlabel('Waist Circumference')
plt.ylabel('Mid Upper Arm Circumference')
plt.title('Waist Circumference and Mid Upper Arm Circumference Regression Model')
print("Regression Score: %.2f " % r2_score(y_test, y_pred))
msq = mean_squared_error(y_test,y_pred)
print("Mean Squared Error :", msq)
# print("R2 score : %.2f" % r2_score(y_test,y_pred))

sns.regplot(x_train, y_train, marker ='+')


In [ ]:
# Model performance
model = sns.regplot(y_test,y_pred, marker='+', color='red')
plt.title("Prediction's scatter Plot Against the Target")
plt.xlabel('feature (y_test)')
plt.ylabel('target (y_pred)')


## ***Functional Model for Subsequent Anthropometric Prediction***

In [ ]:
from datetime import datetime
from datetime import timedelta

from datetime import date

Begin_date = date.today()
End_date = Begin_date + timedelta(days = 100)
next_begin_date = End_date
next_end_date = next_begin_date + timedelta(days= 100)

In [ ]:
# a = new_x_waist
# b = new_y_MUAC
# next_prediction = today + 30

def subs_prediction(a, b):
    for i in range(len(y_test)):
        if today == End_date:
            new_pred = model.predict(new_x_waist)
            print("Latest prediction as of today", today)
            print("New Accuracy Score is: %.2f", r2_score(new_y_MUAC, new_pred)*100)
        else:
            return


if __name__ == '___main__':
    subs_prediction()

In [ ]:
# numpy to be used later
np.corrcoef # correlation coeficient
np.correlate # correlate np arrays

In [ ]:
#another statsmodels that can be used
another_statmodel_lib = sns.regression.statsmodels

#**Preprocessing WASH Features**

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, OneHotEncoder
# WASH_features

# np.corrcoef(bem_wash)
# bem_wash = age_psac.copy()      # Create copy of DataFrame
Age_grp = bem['Age_Group']
# age = Age_grp.fillna(Age_grp.mean()) # Mean imputation
# print(psac_new)

sc = StandardScaler()
le = LabelEncoder()
# #fit label encoder to the toilet types 
# toil_type = bem.iloc[:,13].values
# toilet_type = le.fit_transform(toil_type) # encoded toilet type 

#fit label encoder to the water in toilet variable 
# have_toilet = bem['toilettes?'].values #the values here are Boolean, yet to be preprocesed
# have_toilet = le.fit_transform(have_toilet) #fitted with label encoder now transformed to binary values

toilet_soap = bem['sanitary paper?'].values #the values here are binary: 1s and 0s
toilet_water = bem['water in toilet?'].values # the values here are binary: 1s and 0s and it's an array

# **For Logistic Regression**: Wash and health

In [ ]:
#Questions here is that does, sanitation and hygiene have influence or impact on the state of health in this communities despite the chemotherapy regime?
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
Toilet_type = bem.iloc[:,13] #this contains both tolet type
toilet = bem.iloc[:,12].values
Toilet_type = le.fit_transform(Toilet_type)
sanitation = bem.iloc[:,[14,15]].values # this contains water and tissue paper in the toilet
# le.fit_transform(Toilet)
# # Plotting toilet presence, toilet type against water presence in toilet soap.
# sns.regplot(Toilet, sanitation, color="yellow")
# plt.title('Toilet Type, Water Sanitation and hygiene Regression plot')
# plt.xlabel('Having/Not having a Toilet/Toilet Type')
# plt.ylabel('Have/Do not have water or Soap in the Toilet')
# sns.scatterplot(Toilet,sanitation)
# sns.boxplot(Toilet,sanitation, color='red')

# **BMI and STH**: LinReg

In [ ]:
from locale import normalize
import statsmodels.formula.api as sm
# correlation between having a toilet and body mass index
new_bmi = np.array(bmi) # changed the body mass index into a numpy array
sth = bem.iloc[:,[21,22,23]].values
sth_x = sth[:,np.newaxis,2]
sth_z = sth[:, np.newaxis,1]
model = lr.fit(sth_x,new_bmi)
md = sm.ols('new_bmi ~ sth_x', data=bem).fit()
m = sm.ols('new_bmi ~ sth_z', data=bem).fit()
print(md.summary())
# print(m.summary())

# **Body Mass Index, MUAC and Age**

In [ ]:
df_psac = pd.DataFrame(bem['psac'])
df_sac = pd.DataFrame(bem['sac'])
df_sac.describe()

# **Soil Transmitted Helminthiasis and Anemia status**

**Preprocessing and Linear Regression  Modelling**

In [ ]:
#Urinalysis
urine_parameters = bem.iloc[:,26-29].values.reshape(-1,1)
haem = bem.iloc[:, 18].values
haem = le.fit_transform(haem) #transformed into integer from prtly string and int
urine_parameters = mms.fit_transform(urine_parameters)
# plt.scatter(urine_parameters, haem)
# sns.regplot(urine_parameters, haem)

In [ ]:
from numpy.ma.core import correlate
#STHs
sth = bem[['Trichuris eggcount','Hookworm eggcount','Ascaris eggcount','PCV2']]
# sth = mms.fit_transform(sth).reshape(1,-2)
# pcv = mms.fit_transform(bem.iloc[:,34].values.reshape(1,-2))
#sth against pcv
# plt.scatter(sth, pcv)
# both = [[sth,pcv]]
# sth
# subjecting sth and to linear regression model
x = features = sth.iloc[:,0-2].values.reshape(-1,1) #These are the STHs from line 3 above
y = target = sth.iloc[:,3].values.reshape(-1,1) # this is the packed cell volume 2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

xsc = sc.fit_transform(x)
ysc = sc.fit_transform(y)

x_train, x_test, y_train,y_test = train_test_split(xsc,y, test_size=0.2, random_state=1)
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
model = lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)


sns.regplot(x_train,y_train, marker='*', color='green')
sns.regplot(x_test,y_test,color = 'red')
print('regression coeficient : \n', lr.coef_)
print('mean sqared error: %2f' %mean_squared_error(y_test,y_pred))
print('Model Score:', model.score(x_test,y_test))

In [ ]:
import statsmodels.formula.api as sm
model1 = sm.ols(formula = 'y ~ x', data=bem).fit()
print(model1.summary())

In [ ]:
# separate the conditions: toilet conditions, health conditions and wateravailability of ouake from Bembereke
#use support vector machine and ensemble models for the final prediction.
# use new_bmi 


# ***Linear Regression Model: Biometrics and Anemia status among ***

## **Preschool-Age Children Bembereke and Ouake**

In [ ]:
# psac_df

In [ ]:
# psac_sthand_pcv = psac_df.iloc[:,[30,31,32,33]].values #This is an array of sth and pcv results
# psp = psac_sthand_pcv
# psp

In [ ]:
psac_df = bem[bem.Age_Group != 'sac'] # This sorts our all data that has to do with PSAC
psac_agrp = psac_df.iloc[:,4].values # this contains only the 5th column which is the Age_Group variable.
# plt.hist(psac_agrp)

psth_only = psac_df.iloc[:,[30,31,32]].values #Soil Transmitted helminthes Only Ascaris Hookworm and Trichuris
psth_X = psth_only[:, np.newaxis,2] #values have been sliced.

psac_biometrics = psac_df[['height(cm)','MUAC(cm)','weight(kg)']].values  #PSAC Biometrics 'target'.
pb = psac_biometrics #PSAC Biometrics
pb_Y = pb[:, np.newaxis,2]
# psac_sthand_pcv = psac_df.iloc[:,[30,31,32,33]].values #This is an array of sth and pcv results
# psp = psac_sthand_pcv.reshape(-1,2)
# psth_df.size
# pb.size
#subjecting psac biometrics and infection status to statsmodels
# psac_df.iloc[:,21].values #for ascaris eggcount
model_psac = sm.ols(formula='pb_Y ~ psth_X', data=psac_df).fit()
model_psac.summary()


In [ ]:
# Linear Regression Plot: PSAC Biometrics and STH
x = psth_only.reshape(278,1)
y = pb

# psac_x_train = x[:-20]
# psac_x_test = x[-20:]
# y_train = y[:-20]
# y_test = y[-20:]
# model3 = lr.fit(x_train,y_train)

x_train,x_test, y_train,y_test = train_test_split(x, y, test_size=0.2, random_state=1)

model3 = lr.fit(x_train,y_train)
y_pred = lr.predict(x_test)
# print('Model Accuracy %f' %2f(y_test,y_pred))
# plt.scatter(x_train,y_train, color='red')
# sns.regplot(x_train, y_train, color='green')
Training = plt.scatter(x_train,y_train, color='blue')
regplot = plt.plot(x_train, lr.predict(x_train), color='green')


plt.title('Fig:Multiple Linear Regression Plot: Biometrics VS STH')
plt.xlabel('Soil Transmitted Helminthiasis')
plt.ylabel('Biometrics (Height,Weight,MUAC,Waist Circumference)')
plt.legend(['regplot','Training'])
print('mean sqared error: %2f' %mean_squared_error(y_test,y_pred))
print('regression coeficient : \n', lr.coef_)
model3.score(x_test,y_test)

In [ ]:
Test = plt.scatter(x_test,y_test, color="red")
plt.plot(x_train, lr.predict(x_train), color='green')
plt.legend(['regplot','Testing'])
print('The model accuracy score is:', model3.score(x_test,y_test)*100,'%')

In [ ]:
from sklearn.linear_model import LogisticRegression 
logreg = LogisticRegression()
# what are the determining factors behind having or not having toilet

x = toilet = bem.iloc[:,12].values
x = le.fit_transform(x).reshape(-1,1) # participants ot do not have toilet

y = bem.iloc[:,21-23].values.reshape(-1,1) #sths i.e ascaris, trichuris, hookworm

x_train, x_test, y_train,y_test = train_test_split(x,y, test_size=0.2 , random_state=95)
model5 = logreg.fit(x_train,y_train)
y_pred = model.predict(x_test) #prediction of average sth infection intensity among psac and sac in ouake and bembereke
print('The Model prediction score is:', model5.score(x_test,y_test)*100)
print('The mean squared error is:', mean_squared_error(y_test,y_pred))

plt.scatter(x_train,y_train, marker='+', color='green')
#prediction probability
probability_prediction  = model5.predict_proba(x_test) # shows the probaility of each participant getting infected with either of the sth
model5.predict_log_proba(x_test) #logarithmica probability of being infected with either of the sth
mean_probability = probability_prediction.mean() #mean probability of having either of the sth
print('Mean probability of having either of STH is:', (mean_probability)*100,'%')

from sklearn.metrics import accuracy_score,r2_score
print('Accuracy score is', r2_score(y_test,y_pred)) #R-Squared score i.e. regression score.
plt.xlabel('Have Toilet or Not') #features label
plt.ylabel('Either of STH') #Target label
plt.title('Logistic Regression Plot: Having Toilet Against STH Infection')
stats_model5 = sm.ols(formula='y ~ x', data=bem).fit()
stats_model5.summary()
# from the available evidences, it can be concluded that having or not having toilet does not significantly determine been infected with STH.

In [ ]:
#Logistic regression for classification of parent occupation, sex, toilet
#  In linreg, y = mx + b while in logistic regression, y = 1/1 + e^-(mx + b)
# STH infection and sex
new_x = bem.iloc[:,[21,22,23]].values.reshape(-3,3) #21 - ascaris, #22-Trichuris, #23- 
new_y = bem.iloc[:,5].values.reshape(-1,1)


#Does Age and BMI have anything to do with STH

new_x.size
# plt.scatter(new_x,new_y)